In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 1.6 MB/s eta 0:00:0000:0100:01


In [ ]:
import mysql.connector
import pandas as pd

# Ensure database connection and creation
def create_database():
    """Ensures that the MySQL database exists."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123"
    )
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS data_engineering_books")
    conn.commit()
    cursor.close()
    conn.close()

# Ensure table creation
def create_table():
    """Ensures that the 'books' table exists with the correct schema."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123",
        database="data_engineering_books"
    )
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS books (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255),
            author VARCHAR(255),
            publication_date VARCHAR(50),
            rating VARCHAR(10),
            price VARCHAR(50)
        )
    """)
    
    conn.commit()
    cursor.close()
    conn.close()

# Load data from the CSV file and handle NaN values
def load_data():
    """Loads scraped data from CSV, handles missing values, and inserts into MySQL."""
    df = pd.read_csv("amazon_data_engineering_books.csv")

    # Fill NaN values with "Unknown" for text fields, "Not Available" for price/publication date
    df.fillna({
        'Title': 'Unknown',
        'Author': 'Unknown',
        'Publication Date': 'Not Available',
        'Rating': 'No Rating',
        'Price': 'Not Available'
    }, inplace=True)

    conn = None
    cursor = None

    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Aliza.123",
            database="data_engineering_books"
        )
        cursor = conn.cursor()

        # Insert data into MySQL
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO books (title, author, publication_date, rating, price)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['Title'], row['Author'], row['Publication Date'], row['Rating'], row['Price']))
        
        conn.commit()
        print("Data successfully inserted into MySQL!")

    except mysql.connector.Error as db_error:
        print(f"Database error: {db_error}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Extract and sort data
def query_books():
    """Executes the SQL query to extract and sort books based on rating."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aliza.123",
        database="data_engineering_books"
    )
    cursor = conn.cursor()

    query = """
        SELECT title, author, rating
        FROM books
        ORDER BY rating ASC;
    """
    cursor.execute(query)
    results = cursor.fetchall()

    df_result = pd.DataFrame(results, columns=["Title", "Author", "Price", "Publication Date"])
    
    print("\nTop-rated Data Engineering Books:\n")
    print(df_result)

    cursor.close()
    conn.close()

# Execute functions
create_database()
create_table()
load_data()
query_books()


Data successfully inserted into MySQL!

Top-rated Data Engineering Books:

                                                Title  \
0   "Cracking the Data Engineering Interview: Land...   
1   "Cracking the Data Engineering Interview: Land...   
2   "Cracking the Data Engineering Interview: Land...   
3   "Cracking the Data Engineering Interview: Land...   
4   "Data Engineering with Python: Work with massi...   
5   "Data Engineering with Python: Work with massi...   
6   "Data Engineering with AWS: Acquire the skills...   
7   "Data Engineering with AWS: Acquire the skills...   
8   "Data Engineering with Alteryx: Helping data e...   
9   "Data Engineering with AWS: Acquire the skills...   
10  "Data Engineering with AWS: Acquire the skills...   
11  "Data Engineering with Alteryx: Helping data e...   
12  "Data Engineering with Databricks Cookbook: Bu...   
13  "Data Engineering with Databricks Cookbook: Bu...   
14                        "Data Engineering on Azure"   
15           